In [1]:
import numpy as np
import pandas as pd
import sys
import re
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
model_path = '/home/lambert/Git/Titanic/Titanic_model/'

/home/lambert/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
import re
def get_title(name):
    if pd.isnull(name):
        return 'Null'
    title_search = re.search('([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return 'None'

In [4]:
titles = {'mr':1, 'mrs':2, 'mme':2, 'ms':3, 'miss':3, 'mlle':3, 'don':4, 'sir':4, 'jonkheer':4,
          'major':4, 'col':4, 'dr':4, 'master':4, 'capt':4, 'dona':5, 'lady':5, 'countess':5,
         'rev':6}
df_train['Title'] = df_train['Name'].apply(lambda name: titles.get(get_title(name)))
df_train['Honor'] = df_train['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)
df_test['Title'] = df_test['Name'].apply(lambda name: titles.get(get_title(name)))
df_test['Honor'] = df_test['Title'].apply(lambda title: 1 if title ==4 or title == 5 else 0)

In [5]:
# 1. Data clearning
# df_train.Age.fillna(fill_df_Age, inplace=True)
# df_test.Age.fillna(fill_df_Age, inplace=True)
for i in range(1,7):
    a = df_train[(df_train.Age.isnull()) & (df_train.Title==i)]
    df_train.Age.iloc[a.index] = df_train.Age[df_train.Title==i].median()
    b = df_test[(df_test.Age.isnull()) & (df_test.Title==i)]
    df_test.Age.iloc[b.index] = df_train.Age[df_train.Title==i].median()

# Cabin - fill nan with 'fill' and select first cabin
df_train.Cabin.fillna('fill',inplace=True)
df_train.Cabin = df_train.Cabin.map(lambda x:x.split(' ')[0])
df_test.Cabin.fillna('fill', inplace=True)
df_test.Cabin = df_test.Cabin.map(lambda x:x.split(' ')[0])

# Sex - male: 1, female: 1
df_train.Sex.replace({'male':1, 'female':0}, inplace=True)
df_test.Sex.replace({'male':1, 'female':0}, inplace=True)

df_train = df_train.dropna(axis=0, how='any')
df_test.Fare.fillna(df_test.Fare.mean(), inplace=True)

# Create a 'Deceased' column for sconed class
df_train['Deceased'] = df_train.Survived.apply(lambda s: int(not s))

/home/lambert/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
# 2. Select the features
sFeatures = ['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',             
 'Cabin',             
 'Embarked',
 'Title',
 'Honor']

X = df_train[sFeatures]
y = df_train[['Survived','Deceased']]
y = y.values.reshape(len(y),2)
X_test = df_test[sFeatures]

In [7]:
# 3. Label Encoder
# X_test has elements that X doesn't have
# create a Cabin_labels cover X and X_test
X_Cabin_unique = X['Cabin'].unique()
X_lack = X_test.Cabin[X_test['Cabin'].isin(X_Cabin_unique)==0].values
Cabin_labels = np.append(X_Cabin_unique, X_lack)
# print(X_Cabin_unique.shape, X_Cabin_unique)
# print(Cabin_labels.shape, Cabin_labels)

le_Cabin = LabelEncoder().fit(Cabin_labels)
le_Embarked = LabelEncoder().fit(np.array(X['Embarked'].tolist()))

X.loc[:,'Cabin'] = le_Cabin.transform(X['Cabin'])
X.loc[:,'Embarked'] = le_Embarked.transform(X['Embarked'])
X_test.loc[:,'Cabin'] = le_Cabin.transform(X_test['Cabin'])
X_test.loc[:,'Embarked'] = le_Embarked.transform(X_test['Embarked'])
X = X.astype(np.float32)
y = y.astype(np.float32)

/home/lambert/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
# Validation
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [9]:
X_val[0:10]

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title,Honor
316,2.0,0.0,24.0,1.0,0.0,26.000000,180.0,2.0,2.0,0.0
768,3.0,1.0,30.0,1.0,0.0,24.150000,180.0,1.0,1.0,0.0
275,1.0,0.0,63.0,1.0,0.0,77.958298,144.0,2.0,3.0,0.0
513,1.0,0.0,54.0,1.0,0.0,59.400002,180.0,0.0,2.0,0.0
593,3.0,0.0,21.0,0.0,2.0,7.750000,180.0,1.0,3.0,0.0
15,2.0,0.0,55.0,0.0,0.0,16.000000,180.0,2.0,2.0,0.0
111,3.0,0.0,14.5,1.0,0.0,14.454200,180.0,0.0,3.0,0.0
211,2.0,0.0,35.0,0.0,0.0,21.000000,180.0,2.0,3.0,0.0
878,3.0,1.0,30.0,0.0,0.0,7.895800,180.0,2.0,1.0,0.0
374,3.0,0.0,3.0,3.0,1.0,21.075001,180.0,2.0,3.0,0.0


### Tensor Graph

In [10]:
def next_batch(batch, X, y):
    num = np.arange(len(X))
    np.random.shuffle(num)
    select = num[0:batch]
    x_batch = X.iloc[select]
    y_batch = y[select]
    
    return(x_batch, y_batch)

In [11]:
batch_size = 200

def optimize(num_iterations):
    total_loss = 0
    for i in range(num_iterations):
        x_batch, y_batch = next_batch(batch_size, X_train_data, y_train_data)
        feed_dict_train = {X_place: x_batch, y_place: y_batch}
        _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
        total_loss += loss
        print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
    print ('Training complete!')

In [12]:
# Tensorflow variables
# with tf.device('/device:GPU:0'):
features= len(sFeatures)
X_place = tf.placeholder(tf.float32, shape=[None,features])
y_place = tf.placeholder(tf.float32, shape=[None,2])
W = tf.Variable(tf.random_normal([features, 2]), name='weights')
b = tf.Variable(tf.zeros(2), name='bias')
logits = tf.matmul(X_place, W) + b
y_pred = tf.nn.softmax(logits)

In [13]:
# Cross Entropy
# with tf.device('/device:GPU:0'):
learning_rate = 0.001
cross_entropy = -tf.reduce_sum(y_place * tf.log(y_pred + 1e-10), axis=1)
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_train)
cost = tf.reduce_mean(cross_entropy)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
saver = tf.train.Saver()

In [14]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [15]:
# # training starting option 1
# for epoch in range(10000):
#     total_loss = 0.
#     x_batch, y_batch = next_batch(batch_size, X_train, y_train)
#     feed_dict_train = {X_place: x_batch, y_place: y_batch}
#     _, loss = sess.run([train_op, cost], feed_dict=feed_dict_train)
#     total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [16]:
# sess.close()

In [18]:
# training starting option 1 with CPU
# for epoch in range(100):
#     total_loss = 0.
#     for i in range(len(X_train)):
#         feed = {X_place: X_train, y_place: y_train}
#         _, loss = sess.run([train_op, cost], feed_dict=feed)
#         total_loss += loss
#     print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
# #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
# #     print('predict: {:.6f}'.format(y_pred))
# print ('Training complete!')

In [19]:
# training starting option 1 with GPU
# with tf.device('/device:GPU:0'):
start = time.time()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device('/device:GPU:0'):
        sess.run(tf.global_variables_initializer())
        for epoch in range(10000):
            total_loss = 0.
            for i in range(len(X_train)):
                feed = {X_place: X_train, y_place: y_train}
                _, loss = sess.run([train_op, cost], feed_dict=feed)
                total_loss += loss
            print('Epoch: {:04d}, total loss={:.9f}'.format(epoch+1, total_loss))
        #     print('Epoch: {:04d}, total loss={:.9f}, entropy: {}'.format(epoch+1, total_loss, cross_entropy))
        #     print('predict: {:.6f}'.format(y_pred))
        print ('Training complete!')

        feed_dict_train = {X_place: X_train}
        prediction = sess.run(y_pred, feed_dict=feed_dict_train)
        correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
        # evaluate the accuracy
        accuracy = np.mean(correct)
        print('Accuracy on validation: {:2.2%}'.format (accuracy))
    save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
end = time.time()
print('time spend: {:.4f} sec'.format(end-start))

Epoch: 0001, total loss=5730.189180374
Epoch: 0002, total loss=5725.331882477
Epoch: 0003, total loss=5722.466038704
Epoch: 0004, total loss=5721.388611794
Epoch: 0005, total loss=5720.556382179
Epoch: 0006, total loss=5719.813512802
Epoch: 0007, total loss=5719.103746414
Epoch: 0008, total loss=5718.392380714
Epoch: 0009, total loss=5717.642710686
Epoch: 0010, total loss=5716.773367882
Epoch: 0011, total loss=5715.406972885
Epoch: 0012, total loss=5711.482224464
Epoch: 0013, total loss=5699.442359924
Epoch: 0014, total loss=5683.528650284
Epoch: 0015, total loss=5673.431324005
Epoch: 0016, total loss=5664.011835098
Epoch: 0017, total loss=5655.567133904
Epoch: 0018, total loss=5648.767115593
Epoch: 0019, total loss=5644.528802872
Epoch: 0020, total loss=5642.563036919
Epoch: 0021, total loss=5640.635513306
Epoch: 0022, total loss=5470.453576088
Epoch: 0023, total loss=4621.838045955
Epoch: 0024, total loss=2783.480587602
Epoch: 0025, total loss=2763.659899354
Epoch: 0026, total loss=2

Epoch: 0212, total loss=1169.360448897
Epoch: 0213, total loss=1169.641302764
Epoch: 0214, total loss=1152.810614824
Epoch: 0215, total loss=1157.511389256
Epoch: 0216, total loss=1180.615055323
Epoch: 0217, total loss=1153.132261336
Epoch: 0218, total loss=1204.787439942
Epoch: 0219, total loss=1162.615954638
Epoch: 0220, total loss=1118.269417405
Epoch: 0221, total loss=1177.085435748
Epoch: 0222, total loss=1148.802758574
Epoch: 0223, total loss=1137.499235153
Epoch: 0224, total loss=1150.629481494
Epoch: 0225, total loss=1195.330846190
Epoch: 0226, total loss=1152.471084177
Epoch: 0227, total loss=1184.306220412
Epoch: 0228, total loss=1158.681581199
Epoch: 0229, total loss=1206.101761520
Epoch: 0230, total loss=1121.622428715
Epoch: 0231, total loss=1101.375083983
Epoch: 0232, total loss=1177.909068823
Epoch: 0233, total loss=1174.410443306
Epoch: 0234, total loss=1171.053434908
Epoch: 0235, total loss=1199.956743181
Epoch: 0236, total loss=1129.791115463
Epoch: 0237, total loss=1

Epoch: 0423, total loss=1123.081479490
Epoch: 0424, total loss=1096.079949379
Epoch: 0425, total loss=1171.019536555
Epoch: 0426, total loss=1124.483633935
Epoch: 0427, total loss=1207.420855701
Epoch: 0428, total loss=1120.152893722
Epoch: 0429, total loss=1127.104122639
Epoch: 0430, total loss=1156.872230113
Epoch: 0431, total loss=1155.462266505
Epoch: 0432, total loss=1153.189208865
Epoch: 0433, total loss=1192.571684957
Epoch: 0434, total loss=1101.777176142
Epoch: 0435, total loss=1085.082429171
Epoch: 0436, total loss=1115.089596152
Epoch: 0437, total loss=1123.891366959
Epoch: 0438, total loss=1161.186831832
Epoch: 0439, total loss=1135.074215889
Epoch: 0440, total loss=1128.357885540
Epoch: 0441, total loss=1150.338640094
Epoch: 0442, total loss=1148.951217592
Epoch: 0443, total loss=1093.164834738
Epoch: 0444, total loss=1183.651553571
Epoch: 0445, total loss=1093.088959754
Epoch: 0446, total loss=1160.675934732
Epoch: 0447, total loss=1135.624220014
Epoch: 0448, total loss=1

Epoch: 0634, total loss=1193.520878136
Epoch: 0635, total loss=1026.732347250
Epoch: 0636, total loss=1115.300511360
Epoch: 0637, total loss=1094.975049138
Epoch: 0638, total loss=1137.087134063
Epoch: 0639, total loss=1138.655432761
Epoch: 0640, total loss=1174.204688847
Epoch: 0641, total loss=1079.118419349
Epoch: 0642, total loss=1122.020341277
Epoch: 0643, total loss=1118.526641011
Epoch: 0644, total loss=1143.460666656
Epoch: 0645, total loss=1085.239416718
Epoch: 0646, total loss=1129.906017661
Epoch: 0647, total loss=1179.180070043
Epoch: 0648, total loss=1063.932431102
Epoch: 0649, total loss=1155.152034700
Epoch: 0650, total loss=1058.028729498
Epoch: 0651, total loss=1115.278043509
Epoch: 0652, total loss=1187.080398977
Epoch: 0653, total loss=1110.362271011
Epoch: 0654, total loss=1077.489638209
Epoch: 0655, total loss=1106.342286110
Epoch: 0656, total loss=1123.900818586
Epoch: 0657, total loss=1151.411461532
Epoch: 0658, total loss=1067.705122530
Epoch: 0659, total loss=1

Epoch: 0845, total loss=1068.773012280
Epoch: 0846, total loss=1067.302997172
Epoch: 0847, total loss=1105.193027139
Epoch: 0848, total loss=1124.529604614
Epoch: 0849, total loss=1205.242416739
Epoch: 0850, total loss=1023.687408805
Epoch: 0851, total loss=1086.345440805
Epoch: 0852, total loss=1131.951997280
Epoch: 0853, total loss=1073.198286474
Epoch: 0854, total loss=1092.949672103
Epoch: 0855, total loss=1162.883733511
Epoch: 0856, total loss=980.134121835
Epoch: 0857, total loss=1163.156411707
Epoch: 0858, total loss=1141.524426818
Epoch: 0859, total loss=1083.084898114
Epoch: 0860, total loss=1043.418195248
Epoch: 0861, total loss=1135.036172152
Epoch: 0862, total loss=1055.704219937
Epoch: 0863, total loss=1102.266646981
Epoch: 0864, total loss=1172.641535819
Epoch: 0865, total loss=1125.354490161
Epoch: 0866, total loss=1141.571334004
Epoch: 0867, total loss=1037.343123794
Epoch: 0868, total loss=1065.427988648
Epoch: 0869, total loss=1148.066612065
Epoch: 0870, total loss=10

...

Epoch: 8990, total loss=1040.525273144
Epoch: 8991, total loss=892.685145497
Epoch: 8992, total loss=1027.577395916
Epoch: 8993, total loss=844.520497262
Epoch: 8994, total loss=888.457557440
Epoch: 8995, total loss=958.301880062
Epoch: 8996, total loss=915.729700327
Epoch: 8997, total loss=909.618968248
Epoch: 8998, total loss=940.522662818
Epoch: 8999, total loss=909.803099215
Epoch: 9000, total loss=1004.481163085
Epoch: 9001, total loss=865.583408117
Epoch: 9002, total loss=955.666207433
Epoch: 9003, total loss=932.392623484
Epoch: 9004, total loss=922.944441319
Epoch: 9005, total loss=954.158599079
Epoch: 9006, total loss=962.889015198
Epoch: 9007, total loss=940.838767350
Epoch: 9008, total loss=916.429245949
Epoch: 9009, total loss=974.049829364
Epoch: 9010, total loss=917.522271812
Epoch: 9011, total loss=981.982983589
Epoch: 9012, total loss=875.512264192
Epoch: 9013, total loss=907.728509128
Epoch: 9014, total loss=895.065226912
Epoch: 9015, total loss=999.537710607
Epoch: 90

Epoch: 9205, total loss=913.997192383
Epoch: 9206, total loss=1001.007305264
Epoch: 9207, total loss=966.335534155
Epoch: 9208, total loss=923.263739705
Epoch: 9209, total loss=947.916425347
Epoch: 9210, total loss=940.451172471
Epoch: 9211, total loss=945.070501328
Epoch: 9212, total loss=921.274464190
Epoch: 9213, total loss=879.970128536
Epoch: 9214, total loss=900.723821759
Epoch: 9215, total loss=983.743017793
Epoch: 9216, total loss=955.107646942
Epoch: 9217, total loss=964.464535892
Epoch: 9218, total loss=918.226592600
Epoch: 9219, total loss=893.829060078
Epoch: 9220, total loss=915.439269781
Epoch: 9221, total loss=932.323667586
Epoch: 9222, total loss=931.866433918
Epoch: 9223, total loss=974.166958511
Epoch: 9224, total loss=888.766421497
Epoch: 9225, total loss=960.568400323
Epoch: 9226, total loss=955.049343288
Epoch: 9227, total loss=990.842524886
Epoch: 9228, total loss=919.677231848
Epoch: 9229, total loss=945.919256628
Epoch: 9230, total loss=859.821997523
Epoch: 9231

Epoch: 9421, total loss=859.328202724
Epoch: 9422, total loss=888.973544776
Epoch: 9423, total loss=1007.244788527
Epoch: 9424, total loss=916.159090400
Epoch: 9425, total loss=930.954402924
Epoch: 9426, total loss=990.429784954
Epoch: 9427, total loss=882.765777528
Epoch: 9428, total loss=968.722879767
Epoch: 9429, total loss=902.540598989
Epoch: 9430, total loss=884.401380241
Epoch: 9431, total loss=897.272175968
Epoch: 9432, total loss=982.861073852
Epoch: 9433, total loss=1025.736011267
Epoch: 9434, total loss=953.482092381
Epoch: 9435, total loss=899.854984641
Epoch: 9436, total loss=898.983262777
Epoch: 9437, total loss=930.181516290
Epoch: 9438, total loss=952.539808512
Epoch: 9439, total loss=1067.532442033
Epoch: 9440, total loss=781.030842364
Epoch: 9441, total loss=898.969942749
Epoch: 9442, total loss=955.818487823
Epoch: 9443, total loss=880.225884914
Epoch: 9444, total loss=914.564090133
Epoch: 9445, total loss=981.017072201
Epoch: 9446, total loss=973.852882624
Epoch: 94

Epoch: 9637, total loss=896.846308529
Epoch: 9638, total loss=871.305879056
Epoch: 9639, total loss=972.936231673
Epoch: 9640, total loss=956.162195683
Epoch: 9641, total loss=965.417476952
Epoch: 9642, total loss=919.314783633
Epoch: 9643, total loss=833.786062300
Epoch: 9644, total loss=935.012401164
Epoch: 9645, total loss=909.842466652
Epoch: 9646, total loss=936.273502171
Epoch: 9647, total loss=923.904332995
Epoch: 9648, total loss=1016.050515354
Epoch: 9649, total loss=881.279454887
Epoch: 9650, total loss=940.318347752
Epoch: 9651, total loss=983.882738531
Epoch: 9652, total loss=886.484657943
Epoch: 9653, total loss=1009.037125945
Epoch: 9654, total loss=934.109165192
Epoch: 9655, total loss=867.462099671
Epoch: 9656, total loss=912.728493989
Epoch: 9657, total loss=1002.477017641
Epoch: 9658, total loss=913.784168959
Epoch: 9659, total loss=955.460989833
Epoch: 9660, total loss=878.312801003
Epoch: 9661, total loss=934.720291972
Epoch: 9662, total loss=989.423228920
Epoch: 96

Epoch: 9852, total loss=895.378602922
Epoch: 9853, total loss=901.757978201
Epoch: 9854, total loss=945.894739211
Epoch: 9855, total loss=954.612196982
Epoch: 9856, total loss=945.185637951
Epoch: 9857, total loss=846.900457323
Epoch: 9858, total loss=958.321802378
Epoch: 9859, total loss=919.951222539
Epoch: 9860, total loss=968.871004760
Epoch: 9861, total loss=947.497665763
Epoch: 9862, total loss=904.388877690
Epoch: 9863, total loss=961.892804921
Epoch: 9864, total loss=942.429462433
Epoch: 9865, total loss=918.433229923
Epoch: 9866, total loss=949.444860280
Epoch: 9867, total loss=935.085393667
Epoch: 9868, total loss=869.807556152
Epoch: 9869, total loss=948.249316812
Epoch: 9870, total loss=964.618360221
Epoch: 9871, total loss=929.894417405
Epoch: 9872, total loss=858.179123878
Epoch: 9873, total loss=949.449370265
Epoch: 9874, total loss=934.777602911
Epoch: 9875, total loss=950.837511241
Epoch: 9876, total loss=975.962991595
Epoch: 9877, total loss=900.537342012
Epoch: 9878,

In [ ]:
# save_path = saver.save(sess, model_path+'Titanic_model.ckpt')
# sess.close()

In [21]:
# evaluate the training data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_train = {X_place: X_train}
    prediction = sess.run(y_pred, feed_dict=feed_dict_train)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_train,axis=1))
# evaluate the accuracy
accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 81.98%


In [22]:
# evaluate the validate data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_valid = {X_place: X_val}
    prediction = sess.run(y_pred, feed_dict=feed_dict_valid)
    correct = np.equal(np.argmax(prediction,axis=1), np.argmax(y_val,axis=1))
# evaluate the accuracy

accuracy = np.mean(correct)
print('Accuracy on validation: {:2.2%}'.format (accuracy))

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt
Accuracy on validation: 74.89%


In [30]:
# prediction for test data
with tf.Session() as sess:
    saver.restore(sess, model_path+'Titanic_model.ckpt')
    feed_dict_test = {X_place: X_test}
    prediction = sess.run(y_pred, feed_dict=feed_dict_test)
    y_predict = np.argmax(prediction, axis=1)
    y_predict_1 = np.argmin(prediction, axis=1)
    
# evaluate the accuracy

INFO:tensorflow:Restoring parameters from /home/lambert/Git/Titanic/Titanic_model/Titanic_model.ckpt


In [32]:
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict_1}).set_index('PassengerId')
submission.to_csv('submission_tf_GDO_v3.csv')

In [29]:
y_predict

array([1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,

In [31]:
y_predict_1

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [27]:
# print_accuracy()
# plot_example_errors()
# optimize(num_iterations=10)
# print_accuracy()
# plot_example_errors()
# plot_weights()

In [ ]:
File_Writer=tf.summary.FileWriter(''/Users/lambert/Documents/Python_code/tensorflow_code/Titanic'
                                  ,sess.graph)

In [ ]:
# with tf.device('/device:GPU:0'):
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:

In [ ]:
# allocate as much GPU memory based on runtime allocations
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
# allocate 40% memory of GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config, ...)

In [ ]:
# Automatically choose existing in case specified one doesn't exit
# allow_soft_placement=True
with tf.device('/device:GPU:2'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with allow_soft_placement and log_device_placement set
# to True.
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))

In [ ]:
sess.close()

In [ ]:
sys.argv[2]

In [ ]:
device_lib.list_local_devices()

Logistic Regression

In [ ]:
# Logistic Regression
lr = LogisticRegression(C=1, max_iter=100, solver='lbfgs').fit(X_train, y_train)
yp = lr.predict(X_train)
y_predict = lr.predict(X_test)

passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_knn.csv')
# passengerid = np.array(df_train.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': yp}).set_index('PassengerId')
# submission.to_csv('y_train_predict.csv')

Gradient Boosting Classifier

In [ ]:
# Gradient Boosting Classifier
# gbdt = GradientBoostingClassifier(n_estimators=300, max_depth=5).fit(X_train, y_train)
# y_predict = gbdt.predict(X_test)

# passengerid = np.array(df_test.PassengerId.tolist())
# submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
# submission.to_csv('submission.csv')

Random forests

In [ ]:
# RandomForest
rfc = RandomForestClassifier(n_estimators=200, max_depth=3).fit(X_train, y_train)
y_predict = rfc.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_random.csv')

Stochastic Gradient Descent (SDG)

In [ ]:
# Stochastic Gradient Descent (SGD) easily scale with more than 10^5 samples and features
sgd = SGDClassifier(alpha=0.01, max_iter=1000).fit(X_train, y_train)
y_predict = sgd.predict(X_test)
passengerid = np.array(df_test.PassengerId.tolist())
submission = pd.DataFrame({'PassengerId': passengerid, 'Survived': y_predict}).set_index('PassengerId')
submission.to_csv('submission_sdg.csv')